# Create Dendrogram

This notebook performs hierarchical agglomerative clustering on the _topics_ of a topic model. It is designed to work with only one topic model at a time (see the **Configuration** section to select which model you want to use). The required configurations are the file to which you wish to save the dendgrogram (note that it is an html file, which you can also download and view locally), the distance metric, and the linkage method. If you want to run this notebook on more than 1 model, change these configurations and re-run the notebook or use the `batch_dendrogram` notebook instead. Large models can take several minutes, so be patient.

The default settings will conform closely to the output of pyLDAvis and the scaled view of dfr-browser. For more information on hierarchical agglomerative clustering, distance metrics, and linkage methods, see this module's <a href="README.md">README</a> file.

## INFO

__author__    = 'Scott Kleinman, Lindsay Thomas'  
__copyright__ = 'copyright 2019, The WE1S Project'  
__license__   = 'MIT'  
__version__   = '1.0.0'  
__email__     = 'scott.kleinman@csun.edu'

## Setup

In [ ]:
# Python imports
import os
from pathlib import Path
from IPython.display import display, HTML

# You should not need to change the settings below.
current_dir                    = %pwd
project_dir                    = str(Path(current_dir).parent.parent)
model_dir                      = project_dir + '/' + 'project_data/models'
partials_path                  = os.path.join(current_dir, 'partials')
save                           = True
index_template_path            = 'scripts/index_template.html'
index_filepath                 = 'index.html'

# Import config and scripts
config_path                    = project_dir + '/config/config.py'
%run {config_path}
%run scripts/model.py

display(HTML('<p style="color: green;">Setup complete.</p>'))

## Configuration

**To select the model you want to produce a dendrogram for:** Navigate to the `your_project_name/project_data/models` directory in your project. Note the name of each subdirectory in that folder. Each subdirectory should be called `topicsn1`, where `n1` is the number of topics you chose to model. You should see a subdirectory for each model you produced. To choose which model you would like to produce a dendrogram visualization for, change the value of `selection` in the cell below to the corresponding subdirectory. For example, if you wanted to produce a dendrogram for the 50-topic model you created, change the value of `selection` below to this:

`selection = 'topics50'`

Please follow this format exactly. **Note that you can select only one model to produce a dendrogram for at a time.**

The dendrogram will be saved to the name you set for the `filename` configuration. It should end in `.html`. The file will be saved to the module's `partials` folder. It is often useful to give files names like `topics25-euclidean-single.html` to indicate the number of topics, distance metric, and linkage method used for the cluster analysis. This is especially helpful if you want to run the **Create Index for Multiple Dendrograms** section below. 

Set the distance metric to `euclidean` and `cosine`. The linkage method may be `single`, `complete`, `average`, or `ward`. Ward linkage requires that the distance metric be set to `euclidean`.

Note that a number of advanced configuration options are available. These are detailed in the README file. For details of the various settings, see the <a href="README.md" target="_blank">README</a> file. If you wish to use advanced configurations, add them directly to the `model.cluster()` call in the **Cluster the Model** cell.

**Note:** The default output will only work in a server environment. If you wish to create a standalone version that can be run on a local computer, set `standalone=True`. The disadvantage of this method is that the file will be large, about 3 MB.

In [ ]:
# Save Parameters
selection          = '' # E.g. 'topics25'
filename           = '' # The name of the file to save to. Change for each model if running multiple times, e.g. 'topics25'. 
title              = '' # Give your dendrogram file a title here
distance_metric    = 'euclidean' # 'euclidean' or cosine'
linkage_method     = 'average' # 'single', 'complete', 'average', or 'ward'
orientation        = 'top' # 'top', 'right', 'bottom', or 'left'
height             = 600 # In pixels
width              = 1200 # In pixels
standalone         = False # If True, output does not require a server but will be about 3MB

display(HTML('<p style="color: green;">Configuration complete.</p>'))

## Load Data from the MALLET State File

This instantiates a `Model` object and loads the model's data from the MALLET state file.

In [ ]:
# Get topic state and keys files
topic_state_file, keys_filepath = get_model_data(selection, model_dir)

# Instantiate the Python Object
model = Model(os.path.join(model_dir, topic_state_file), keys_filepath=keys_filepath, partials_path=partials_path,
              title=title, current_dir=current_dir, WRITE_DIR=WRITE_DIR, PORT=PORT)

## Cluster the Model

By default, the cluster wil be saved as an html `div` element in the `partials` folder.

In [ ]:
# Make the partials folder if necessary
make_partials_path(partials_path)

# Start the cluster analysis
model.cluster(distance_metric, linkage_method, filename=filename, orientation=orientation, 
              height=height, width=width, standalone=standalone, save=True)

## Create Web Page for a Single Dendrogram

This cell generates a web page that displays a single dendrogram. This web page will not work without an internet connection. If you wish to download a copy that does not require an internet connection, set the `standalone` configuration to `True` in the **Configuration** section and re-run the cluster analysis.

If you have already produced multiple dendrograms and wish to publish them with an index page, skip to the next section.

In [ ]:
# Create the web page
model.save(partials_path, filename)

## Create Index Page for Multiple Dendrograms

This section will produce an index page allowing you to navigate between multiple dendrogram files, **which you have already created**. Make sure that you configure the settings under **Index File Configuration** below.

**Note:** The default output will only work in a server environment.

### Index File Configuration

The `index_filename` is the name of the index file you wish to produce. The `selection` value is the name of the model you wish to use (one of the subdirectories in your `project_data/models` folder. The `page_title` value is the title you wish to display at the top of the index page.

If you would like to save the index and dendrograms to a zip archive for export, set `zip=True`.

The `source_filenames` must be the same as dendrogram filenames in your `partials` folder.

The `menu_items` and `dendrogram_titles` lists should correspond to the order of the `source_filenames`. The former will appear as menu labels for navigating between dendrograms, and the latter will be titles for each dendrogram.

In [ ]:
# Configure web page settings
index_filename  = '' # E.g. 'topics25-index.html'
selection       = '' # E.g. 'topics25'
page_title      = '' # E.g. '25 Topics'
zip             = True

# Add the source filenames (e.g. 'topics25-euclidean-average.html') for all dendrograms here
source_filenames = []

# Add menu labels (e.g. 'Euclidean Average') in the same order as the filenames
menu_items = []

# Add dendrogram titles (e.g. Euclidean Distance with Average Linkage) in the same order as the filenames
dendrogram_titles = []

display(HTML('<p style="color: green;">Configuration complete.</p>'))

## Generate the Index Page

This cell will display a link to your index page.

In [ ]:
# Import the model functions
%run scripts/model.py

# Get directory paths
current_dir                    = %pwd
project_dir                    = str(Path(current_dir).parent.parent)
model_dir                      = project_dir + '/' + 'project_data/models'
config_path                    = project_dir + '/config/config.py'

# Import the config
%run {config_path}

# Create the index file
create_index_file(index_filename, selection, model_dir, page_title, source_filenames,
                  menu_items, dendrogram_titles, PORT=PORT, zip=zip)